In [1]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")


# Pytorch device specific configuration
# Pytorch Gpu Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [3]:
# import dataset
iris = pd.read_csv(
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
)

# convert from pandas dataframe to tensor
data = torch.tensor(iris[iris.columns[0:4]].values).float()

# transform species to number
labels = torch.zeros(len(data), dtype=torch.long)
# labels[iris.species=='setosa'] = 0 # don't need!
labels[iris.species == "versicolor"] = 1
labels[iris.species == "virginica"] = 2

In [ ]:
# Break the data into batches
## Use scikitlearn train_test_split to split the data
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2
)

## Convert data into pytorch dataset
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

## Create data loder objects
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
    generator=torch.Generator(device),
)
test_loader = DataLoader(
    test_dataset,
    batch_size=test_dataset.tensors[0].shape[0],
    shuffle=True,
    generator=torch.Generator(device),
)

In [ ]:
# Function to create ANN model
def createNewModel(L2_lambda=0.001, lr=0.005):

    # Model Architecture
    ANNIris = nn.Sequential(
        nn.Linear(4, 64), nn.ReLU(), nn.Linear(64, 64), nn.ReLU(), nn.Linear(64, 3)
    )

    # Loss Function
    lossFun = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.SGD(ANNIris.parameters(), lr=lr, weight_decay=L2_lambda)

    # Function return statement
    return ANNIris, lossFun, optimizer

In [6]:
# Train the model
def trainTheModel(ANNIris, lossFun, optimizer, nEpochs=1000):
    
    # Initialize the accuracies and losses as empty array
    trainAcc = []
    testAcc = []
    losses = []

    # For loop over epochs
    for epoch in range(nEpochs):
        # Initialize batchAcc and batchLoss
        batchAcc = []
        batchLoss = []

        # Switch model into train mode
        ANNIris.train()

        # Loop over batches
        for X, y in train_loader:
            
            # Forward pass
            yHat = ANNIris(X)
            loss = lossFun(yHat, y)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compte batch training accuracy and loss
            batchAcc.append(100 * torch.mean((torch.argmax(yHat, axis=1) == y).float()).item())
            batchLoss.append(loss.item())
        
        # Get average training accuracy for this epoch
        trainAcc.append(np.mean(batchAcc))
        losses.append(np.mean(batchLoss))

        # Test accuracy
        ANNIris.eval() # Model Evaluation mode
        X, y = next(iter(test_loader)) # X, y from test loader dataloader
        predlabels = torch.argmax(ANNIris(X), axis=1)
        testAcc.append(100 * torch.mean((predlabels == y).float()).item())

    # Function return statement
    return trainAcc, testAcc, losses

